In [1]:
!pip install -q transformers[torch]
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("Someman/news_nepali")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/15580 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1732 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'article_summary'],
        num_rows: 15580
    })
    test: Dataset({
        features: ['article', 'article_summary'],
        num_rows: 1732
    })
})

In [20]:
dataset = dataset["train"]

In [24]:
dataset[1]["article"]

'काठमाडौं, भदौ २१ सरकारले अनुगमन गरेको विरोधमा तरकारी व्यवसायीले आज काठमाडौंको कालीमाटी फलफूल तथा तरकारी बजारका सबै पसल बन्द गरेका छन् ।हिजो बजारमा सरकारी अनुगमनकर्तालाई केही समयका लागि हिरासतमा राखेका छन् । केही स्थानीय कृषक बाहेक अन्य सबै बिक्रेताहरु । पसल बन्द गरेर बजारमा तरकारी बेच्न बन्द गरेका छन् । प्याज, आलु, कागती र केही हरियो तरकारी भने पुरानो स्टकबाट बजारमा पाइन्छ । मोटर चलाएपछि तरकारीको मूल्य घट्ने र गुणस्तरमा समेत प्रश्न उठ्ने डर बिक्रेताले गरेका छन् । तरकारी बेच्न बन्द गरेका कारण काठमाडौंका धेरै ठाउँमा तरकारीको अभाव भएको छ । जसका कारण उपभोक्ता सबैभन्दा बढी मारमा परेका छन् । तर, अहिले पर्ख र हेरको अवस्थामा रहेकाले सरकारले यो समस्यालाई सम्बोधन गर्न वा कुनै कारबाही गर्न सकेको छैन । सधैँ भीडभाड हुने कालीमाटी तरकारी बजार दिनभर सुनसान रह्यो । मंगलबार.तरकारी बिक्रेताले पनि सबै मिडियाले आफ्नो विरुद्धमा लेखेको भन्दै फोटो/भिडियो क्लिक गर्न मिडियालाई परिसरमा प्रवेश गर्न निषेध गरेका छन् ।अनलाइनखबरका फोटोग्राफरलाई एक विक्रेताले धेरै तस्विर डिलिट गर्न भन्दै धम्की समेत दिएको छ । पसल खुले

In [25]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset["article"][i : i + 1000]

In [4]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [5]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [9]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase()]
)

In [10]:
print(tokenizer.normalizer.normalize_str("कार्यसम्पादनमा प्रदेश र स्थानीय तहको नतिजा कमजोर"))

कार्यसम्पादनमा प्रदेश र स्थानीय तहको नतिजा कमजोर


In [11]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [12]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

In [13]:
tokenizer.pre_tokenizer.pre_tokenize_str("कार्यसम्पादनमा प्रदेश र स्थानीय तहको नतिजा कमजोर")

[('कार्यसम्पादनमा', (0, 14)),
 ('प्रदेश', (15, 21)),
 ('र', (22, 23)),
 ('स्थानीय', (24, 31)),
 ('तहको', (32, 36)),
 ('नतिजा', (37, 42)),
 ('कमजोर', (43, 48))]

In [15]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)
pre_tokenizer.pre_tokenize_str("कार्यसम्पादनमा प्रदेश र स्थानीय, तहको नतिजा! कमजोर")

[('कार्यसम्पादनमा', (0, 14)),
 ('प्रदेश', (15, 21)),
 ('र', (22, 23)),
 ('स्थानीय', (24, 31)),
 (',', (31, 32)),
 ('तहको', (33, 37)),
 ('नतिजा', (38, 43)),
 ('!', (43, 44)),
 ('कमजोर', (45, 50))]

In [16]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [26]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [27]:
encoding = tokenizer.encode("२३ दिन हिँडेर काठमाडौं आइपुगे मिटरब्याज पीडित (तस्वीरहरु)")
print(encoding.tokens)

['२३', 'दिन', 'हिँडेर', 'काठमाडौं', 'आइपुग', '##े', 'मिटर', '##ब्या', '##ज', 'पीडित', '(', 'तस्', '##वीर', '##हरु', ')']


In [28]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [29]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [35]:
encoding = tokenizer.encode("झापाको काकडभिट्टा र कञ्चनपुरको महेन्द्रनगरबाट पैदल हिँडेका मिटरब्याज पीडित २३ दिनपछि काठमाडौं आइपुगेका छन् । बुधबार साँझ उनीहरु पश्चिमी नाका थानकोटबाट काठमाडौं प्रवेश गरेका हुन् । मिटरब्याजविरुद्ध किसान मजदुर संघर्ष समितिको समन्वयनमा एउटा समूहले झापाबाट र अर्को समूहले महेन्द्रनगरबाट पैदलयात्रा सुरु गरेको थियो ।")
print(encoding.tokens)

['[CLS]', 'झापाको', 'का', '##कडभि', '##ट्टा', 'र', 'कञ्चनपुरको', 'महेन्द्रनगर', '##बाट', 'पैदल', 'हिँड', '##ेका', 'मिटर', '##ब्या', '##ज', 'पीडित', '२३', 'दिनपछि', 'काठमाडौं', 'आइपुगेका', 'छन्', '।', 'बुधबार', 'साँझ', 'उनीहरु', 'पश्चिमी', 'नाका', 'थानकोट', '##बाट', 'काठमाडौं', 'प्रवेश', 'गरेका', 'हुन्', '।', 'मिटर', '##ब्या', '##ज', '##विरुद्ध', 'किसान', 'मजदुर', 'संघर्ष', 'समितिको', 'समन्वय', '##नमा', 'एउटा', 'समूहले', 'झापाबाट', 'र', 'अर्को', 'समूहले', 'महेन्द्रनगर', '##बाट', 'पैदल', '##यात्रा', 'सुरु', 'गरेको', 'थियो', '।', '[SEP]']


In [33]:
encoding = tokenizer.encode("This is the first sentence. This is the second sentence.")
print(encoding.tokens)

['[CLS]', 't', '##hi', '##s', 'i', '##s', 't', '##h', '##e', 'fi', '##r', '##s', '##t', 's', '##en', '##te', '##n', '##c', '##e', '.', 't', '##hi', '##s', 'i', '##s', 't', '##h', '##e', 's', '##ec', '##on', '##d', 's', '##en', '##te', '##n', '##c', '##e', '.', '[SEP]']


In [34]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [36]:
tokenizer.decode(encoding.ids)

'झापाको काकडभिट्टा र कञ्चनपुरको महेन्द्रनगरबाट पैदल हिँडेका मिटरब्याज पीडित २३ दिनपछि काठमाडौं आइपुगेका छन् । बुधबार साँझ उनीहरु पश्चिमी नाका थानकोटबाट काठमाडौं प्रवेश गरेका हुन् । मिटरब्याजविरुद्ध किसान मजदुर संघर्ष समितिको समन्वयनमा एउटा समूहले झापाबाट र अर्को समूहले महेन्द्रनगरबाट पैदलयात्रा सुरु गरेको थियो ।'

In [37]:
tokenizer.save("tokenizer.json")